VGG19 NN

In [9]:
import os
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from tensorflow import keras
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from matplotlib import cm
from PIL import Image, ImageOps
from keras import backend as K
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, InputLayer, MaxPooling2D, AveragePooling2D
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras import layers, models, Model
from keras.applications.vgg19 import VGG19

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip ./drive/MyDrive/data/mrdata.zip

Archive:  ./drive/MyDrive/data/mrdata.zip
   creating: mrdata/
   creating: mrdata/10/
   creating: mrdata/10/depth/
  inflating: mrdata/10/depth/11.29.24.441.png  
  inflating: mrdata/10/depth/11.29.24.554.png  
  inflating: mrdata/10/depth/11.29.24.663.png  
  inflating: mrdata/10/depth/11.29.24.772.png  
  inflating: mrdata/10/depth/11.29.24.883.png  
  inflating: mrdata/10/depth/11.29.24.997.png  
  inflating: mrdata/10/depth/11.29.25.104.png  
  inflating: mrdata/10/depth/11.29.25.210.png  
  inflating: mrdata/10/depth/11.29.25.322.png  
  inflating: mrdata/10/depth/11.29.25.433.png  
  inflating: mrdata/10/depth/11.29.25.543.png  
  inflating: mrdata/10/depth/11.29.25.651.png  
  inflating: mrdata/10/depth/11.29.25.764.png  
  inflating: mrdata/10/depth/mirror11.29.24.405.png  
  inflating: mrdata/10/depth/mirror11.29.24.489.png  
  inflating: mrdata/10/depth/mirror11.29.24.565.png  
  inflating: mrdata/10/depth/mirror11.29.24.646.png  
  inflating: mrdata/10/depth/mirror11.29.24

In [5]:
W = 512
H = 424
max_value_for_normalization = -1

def make_df(path_to_xlsx, path_to_folder):
    if not os.path.exists(path_to_xlsx):
        print('[Err] Fail xlsx path')
        return
    if not os.path.exists(path_to_folder):
        print('[Err] Fail folder path')
        return

    df = pd.read_excel(path_to_xlsx)
    df = df.fillna(0)
    df['Body length (cm)'] = df['Body length (cm)'].astype(int)
    df['Chest girth (cm)'] = df['Chest girth (cm)'].astype(int)
    df['Withers height (cm)'] = df['Withers height (cm)'].astype(int)
    df['Chest depth (cm)'] = df['Chest depth (cm)'].astype(int)
    df['Chest width (cm)'] = df['Chest width (cm)'].astype(int)
    df['Live weight (kg)'] = df['Live weight (kg)'].astype(int)
    df.drop(
        columns=['Frame', 'Content', 'Number of pig', 'Left', 'Right', 'Quality', 'Transform\nT_right_to_left', 'Sex', 'Age (days)'], inplace=True)
    
    df = df.iloc[2:]
    df['image'] = None
    for index, row in df.iterrows():
        path = f'{path_to_folder}/{index}/depth'
        if not os.path.exists(path):
            continue
        # print(path)
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        print(f'For index {index} count {len(files)} files')
        for file in files:
            raw_image = cv2.imread(f'{path}/{file}', cv2.CV_16U).astype(np.float32)
            raw_image = cv2.resize(raw_image, (W, H))
            raw_image = np.repeat(raw_image[..., np.newaxis], 3, -1)
            raw_image = raw_image.reshape(-1, W, H, 3)

            global max_value_for_normalization
            if np.amax(raw_image) > max_value_for_normalization:
                max_value_for_normalization = np.amax(raw_image)

            new_data = {
                'image': raw_image,
                'Live weight (kg)': df.iloc[index - 3]['Live weight (kg)'],
                'Body length (cm)': df.iloc[index - 3]['Body length (cm)'],
                'Chest girth (cm)': df.iloc[index - 3]['Chest girth (cm)'],
                'Withers height (cm)': df.iloc[index - 3]['Withers height (cm)'],
                'Chest depth (cm)': df.iloc[index - 3]['Chest depth (cm)'],
                'Chest width (cm)': df.iloc[index - 3]['Chest width (cm)'],
            }

            df = pd.concat([df, pd.DataFrame.from_records([new_data])])

    df['image'] = df['image'] / max_value_for_normalization
    df = df.iloc[16:]
    df = df.reset_index().drop(columns=['index'])
    return df
  
dataframe = make_df('./drive/MyDrive/data/data.xlsx', 'mrdata')
dataframe


For index 3 count 31 files
For index 4 count 32 files
For index 5 count 21 files
For index 6 count 31 files
For index 7 count 26 files
For index 8 count 39 files
For index 9 count 27 files
For index 10 count 27 files
For index 11 count 24 files
For index 12 count 27 files
For index 13 count 17 files
For index 14 count 37 files
For index 15 count 22 files
For index 16 count 33 files
For index 17 count 22 files


,Live weight (kg),Body length (cm),Chest girth (cm),Withers height (cm),Chest depth (cm),Chest width (cm),image
0,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
1,117,114,117,55,37,32,"[[[[0. 0. 0.], [0.285125 0.285125 0.285125], [..."
2,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
3,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
4,117,114,117,55,37,32,"[[[[0. 0. 0.], [0.935375 0.935375 0.935375], [..."
...,...,...,...,...,...,...,...
411,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
412,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0.1126..."
413,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
414,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."


In [6]:
columns_features_all = ['Live weight (kg)', 'Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
columns_features_kg = ['Live weight (kg)']
columns_features_sm = ['Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
# column_img = ['image']

In [7]:
vgg19_nn = VGG19(include_top = False, weights='imagenet', input_shape=(W, H, 3))
vgg19_nn.trainable = False
# vgg19_nn.summary()

80150528/80134624 [==============================] - 2s 0us/step


ADAM ALL PARAMS

In [25]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(vgg19_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='relu'))
  model.build(input_shape=(None, W, H, 3))

  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/vgg19_nn/best_adam_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'


Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 510.70480, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 510.70480

Epoch 00003: val_loss improved from 510.70480 to 330.92831, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss improved from 330.92831 to 239.23465, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss did not improve from 239.23465

Epoch 00006: val_loss did not improve from 239.23465

Epoch 00007: val_loss improved from 239.23465 to 158.27310, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00008: val_loss did not improve from 158.27310

Epoch 00009: val_loss improved from 158.27310 to 41.73336, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00010: val_loss did not im

In [26]:
evals_models

[[27.594074249267578, 3.4065940380096436, 11.445794105529785],
 [29.019954681396484, 3.423910140991211, 12.38927936553955],
 [27.831911087036133, 3.8704745769500732, 11.510092735290527],
 [34.584747314453125, 4.126193523406982, 15.145613670349121],
 [57.37519836425781, 5.805964946746826, 12.707447052001953],
 [71.86498260498047, 6.579649925231934, 18.97289276123047],
 [22.25821876525879, 3.3741538524627686, 11.061155319213867],
 [25.792375564575195, 3.538245677947998, 13.018954277038574]]

RMSPROP ALL PARAMS

In [27]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(vgg19_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='relu'))
  model.build(input_shape=(None, W, H, 3))

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/vgg19_nn/best_rmsprop_all_params_{count}'
  count = count + 1
  
  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'


rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 2004.87244, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 2004.87244 to 244.99667, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss did not improve from 244.99667

Epoch 00004: val_loss improved from 244.99667 to 143.19740, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss did not improve from 143.19740

Epoch 00006: val_loss did not improve from 143.19740

Epoch 00007: val_loss improved from 143.19740 to 97.58998, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00008: val_loss did not improve from 97.58998

Epoch 00009: val_loss did not improve from 97.58998

Epoch 00010: val_loss improved from 97.58998 to 90.86608, savin

In [28]:
evals_models

[[28.19770622253418, 3.5838208198547363, 12.105219841003418],
 [32.88700866699219, 3.887453079223633, 13.314476013183594],
 [33.77183532714844, 4.438088893890381, 11.998578071594238],
 [36.45861053466797, 4.463229179382324, 15.001062393188477],
 [21.655397415161133, 3.1444127559661865, 9.843149185180664],
 [32.68052673339844, 3.8162057399749756, 16.550846099853516],
 [179.8817596435547, 10.698405265808105, 20.59757423400879],
 [186.77987670898438, 10.964556694030762, 22.5279541015625]]

ADAM KG PARAM

In [29]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(vgg19_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))
  model.build(input_shape=(None, W, H, 3))

  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/vgg19_nn/best_adam_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'


rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 47.38741, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 47.38741

Epoch 00003: val_loss did not improve from 47.38741

Epoch 00004: val_loss did not improve from 47.38741

Epoch 00005: val_loss did not improve from 47.38741

Epoch 00006: val_loss did not improve from 47.38741

Epoch 00007: val_loss did not improve from 47.38741
Epoch 00007: early stopping
'./model/best' -> '/content/drive/MyDrive/data/models/vgg19_nn/best_adam_kg_param_1'
'./model/best/variables' -> '/content/drive/MyDrive/data/models/vgg19_nn/best_adam_kg_param_1/variables'
'./model/best/variables/variables.data-00000-of-00001' -> '/content/drive/MyDrive/data/models/vgg19_nn/best_adam_kg_param_1/variables/variables.data-00000-of-00001'
'./model/best/variables/variables.index' -> '/content/drive/MyDrive/

In [30]:
evals_models

[[47.83559799194336, 5.730053901672363, 5.3193182945251465],
 [60.255126953125, 6.770105361938477, 6.260202407836914],
 [30.238258361816406, 4.854166507720947, 4.56247091293335],
 [36.72499465942383, 5.487157344818115, 5.121697425842285],
 [24.37087059020996, 3.7676663398742676, 3.428373336791992],
 [33.10816955566406, 4.461087703704834, 4.031555652618408],
 [26.571571350097656, 4.54608678817749, 4.273464202880859],
 [37.522735595703125, 5.408145427703857, 5.095114231109619]]

RMSPROP KG PARDM

In [31]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(vgg19_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))
  model.build(input_shape=(None, W, H, 3))

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/vgg19_nn/best_rmsprop_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'


rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 2152.69165, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 2152.69165 to 35.62081, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss did not improve from 35.62081

Epoch 00004: val_loss did not improve from 35.62081

Epoch 00005: val_loss did not improve from 35.62081

Epoch 00006: val_loss did not improve from 35.62081

Epoch 00007: val_loss did not improve from 35.62081

Epoch 00008: val_loss did not improve from 35.62081
Epoch 00008: early stopping
'./model/best' -> '/content/drive/MyDrive/data/models/vgg19_nn/best_rmsprop_kg_param_1'
'./model/best/variables' -> '/content/drive/MyDrive/data/models/vgg19_nn/best_rmsprop_kg_param_1/variables'
'./model/best/variables/variables.index' -> '/content/drive/MyDrive/data/models/

In [33]:
evals_models

[[35.60672378540039, 5.05592679977417, 4.759037971496582],
 [41.978981018066406, 5.627011299133301, 5.257389068603516],
 [25.43536949157715, 4.136418342590332, 3.7818617820739746],
 [27.514400482177734, 3.9916255474090576, 3.6211605072021484],
 [24.694538116455078, 4.0229902267456055, 3.7066760063171387],
 [30.450056076049805, 4.465433120727539, 4.075542449951172],
 [24.542818069458008, 3.6765482425689697, 3.3126933574676514],
 [30.14190101623535, 3.98945951461792, 3.596395492553711]]

ADAM SM PARAMS

In [34]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(vgg19_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='relu'))
  model.build(input_shape=(None, W, H, 3))

  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/vgg19_nn/best_adam_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'


rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 397.38147, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 397.38147 to 392.11893, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss did not improve from 392.11893

Epoch 00004: val_loss improved from 392.11893 to 257.46100, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss improved from 257.46100 to 215.12875, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00006: val_loss improved from 215.12875 to 159.80180, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss improved from 159.80180 to 132.50169, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/b

In [35]:
evals_models

[[127.4964370727539, 10.246307373046875, 23.730806350708008],
 [119.26995849609375, 9.812263488769531, 23.635183334350586],
 [31.649024963378906, 4.2655134201049805, 13.092254638671875],
 [36.575042724609375, 4.52090311050415, 17.23786163330078],
 [20.728313446044922, 2.9250805377960205, 11.09567928314209],
 [32.34491729736328, 3.5035109519958496, 18.84235382080078],
 [30.411752700805664, 3.8651013374328613, 13.109431266784668],
 [35.10905075073242, 4.038186073303223, 15.736411094665527]]

RMSPROP SM PARAMS

In [37]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(vgg19_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='relu'))
  model.build(input_shape=(None, W, H, 3))

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/vgg19_nn/best_rmsprop_sm_params_{count}'
  count = count + 1  

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'


Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 962.75269, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 962.75269 to 528.42456, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss improved from 528.42456 to 281.26804, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 281.26804

Epoch 00005: val_loss did not improve from 281.26804

Epoch 00006: val_loss improved from 281.26804 to 191.73683, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss improved from 191.73683 to 54.68008, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00008: val_loss did not improve from 54.68008

Epoch 00009: val_loss did not improve from 54.68008

Epoch 00010: val_loss did not impr

In [38]:
evals_models

[[51.373939514160156, 5.543203353881836, 15.061492919921875],
 [55.61438751220703, 5.799027919769287, 15.979246139526367],
 [23.362844467163086, 2.885578155517578, 12.366466522216797],
 [34.00010299682617, 3.474729299545288, 17.64072608947754],
 [18.713085174560547, 2.562594413757324, 10.782353401184082],
 [31.585140228271484, 3.2423818111419678, 18.744213104248047],
 [24.825618743896484, 3.0398380756378174, 12.666033744812012],
 [28.01163101196289, 3.0649220943450928, 15.265661239624023]]